In [2]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime, timedelta

In [9]:
from biblioteca_qca import tratamento_prazos_diarios_agendados

In [27]:
tratamento_prazos_diarios('base1302_agendados.xlsx', 'centro_de_custo.xlsx')

In [3]:
hoje = date.today()

In [4]:
df = pd.read_excel('base1402_agendados.xlsx', engine='openpyxl')

c:\Users\victoramarante\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
df.shape

(2111, 66)

In [6]:
bmg
cetelem - suellen
entradas - 21 e 22

SyntaxError: invalid syntax (1311015404.py, line 3)

In [7]:
# dropando colunas que nao serao usadas
df = df.drop(['NPC', 'Número do Órgão', 'Sistema Acompanhamento', 'Responsável Cancelamento',
                          'ID Obrigação', 'Observação ', 'Observação da conclusão', 'Projeto',
                          'Link de Acesso à Audiência'],
                          axis=1)

In [8]:
# criando duas novas colunas vazias
df[' '] = ' '
df[' '] = ' '

In [9]:
df.shape

(2111, 58)

In [10]:
# selecionar o tipo de atividade = diligencia e nao selecionar os protocolos e apagar todas as linhas
df_remove = df[(df['Tipo de Atividade'] == 'Diligência') &
    (df['Sub Tipo Compromisso'].str.contains('Protocolo') == False)]

In [11]:
df = df.drop(df_remove.index)

In [12]:
centro_custo = pd.read_excel('centro_de_custo.xlsx', engine='openpyxl')

In [13]:
df_ajustado = pd.merge(df, centro_custo, on='Célula', how='left')
df_ajustado.drop(['Centro de Custo_x', 'Gestor', 'Diretoria', 'Escritorio'], axis=1, inplace=True)

df_ajustado = df_ajustado[['ID Processo', 'ID Prazo', 'Data Cadastro Prazo', 'Data do Prazo','Hora do Prazo', 'Tipo de Atividade', 'Sub Tipo Compromisso','Parte Cliente', 'Parte Adversa', 'Número do Processo', 'Órgão',
       'Comarca', 'UF', 'Tipo de Processo', 'Sistema Eletrônico','Status do Prazo', 'Data Inicio Compromisso', 'Data Conclusão','Data Protocolo', 'Data Auditoria Protocolo', 'Prazo para Protocolo',
       'Data Prazo Automático', 'Data Revisão', 'Prazo Revisão','Data de Reprovação', 'Data Cancelamento','Motivo do cancelamento do Agendamento', 'Observação de Cancelamento',
       'Fase', 'Estratégia', 'Objeto', 'Sub-Objeto', 'Produto','Adv. Responsável Processo', 'Responsável Cadastro','Responsavel Prazo', 'Responsável Revisão', 'Responsável Conclusão',
       'Responsável Protocolo', 'Responsável Auditoria Protocolo', 'Célula','Centro de Custo_y','Cliente', 'Segmento', 'Forma Abertura', 'ID PA', 'ID Prazo Origem',
       ' ', ' ', 'Tem serviço contratado?', 'Id serviço', 'Tipo','Subtipo do serviço', 'Data da contratação', 'Hora da contratação','Status do serviço', 'Status Sugestão alteração/cancelamento',
       'Vinculado ao Recurso?', 'ID Recurso']]

In [14]:
df_ajustado.rename(columns = {'Centro de Custo_y': 'Centro de Custo'}, inplace=True)

df_final = df_ajustado[(df_ajustado['Centro de Custo'] != 'AMBEV') &
            (df_ajustado['Centro de Custo'] != 'CCB MASSIFICADO') &
            (df_ajustado['Centro de Custo'] != 'Apresentação QCA') &
            (df_ajustado['Centro de Custo'] != 'EQUIPE PARAÍBA') &
            (df_ajustado['Centro de Custo'] != 'MONGERAL') &
            (df_ajustado['Centro de Custo'] != 'MARITIMO E PORTUARIO') &
            (df_ajustado['Centro de Custo'] != ' ')]

In [15]:
# from biblioteca_qca import tratamento_prazos_diarios

# import warnings
# warnings.filterwarnings('ignore')

In [16]:
# tratamento_prazos_diarios(base_agendados='base1302_agendados.xlsx',
#                             base_centro_custo='centro_de_custo.xlsx')

In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2097
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID Processo                             2091 non-null   int64         
 1   ID Prazo                                2091 non-null   int64         
 2   Data Cadastro Prazo                     2091 non-null   datetime64[ns]
 3   Data do Prazo                           2091 non-null   datetime64[ns]
 4   Hora do Prazo                           2091 non-null   object        
 5   Tipo de Atividade                       2091 non-null   object        
 6   Sub Tipo Compromisso                    2091 non-null   object        
 7   Parte Cliente                           2091 non-null   object        
 8   Parte Adversa                           2091 non-null   object        
 9   Número do Processo                      2091 non-nul

In [18]:
df_final.select_dtypes('datetime').columns

Index(['Data Cadastro Prazo', 'Data do Prazo', 'Data Inicio Compromisso',
       'Data Conclusão', 'Data Protocolo', 'Data Auditoria Protocolo',
       'Prazo para Protocolo', 'Data Prazo Automático', 'Data Revisão',
       'Prazo Revisão', 'Data Cancelamento', 'Data da contratação'],
      dtype='object')

In [19]:
colunas_datas = list(df_final.select_dtypes('datetime').columns)

In [20]:
colunas_datas.append('Data de Reprovação')

In [21]:
colunas_datas

['Data Cadastro Prazo',
 'Data do Prazo',
 'Data Inicio Compromisso',
 'Data Conclusão',
 'Data Protocolo',
 'Data Auditoria Protocolo',
 'Prazo para Protocolo',
 'Data Prazo Automático',
 'Data Revisão',
 'Prazo Revisão',
 'Data Cancelamento',
 'Data da contratação',
 'Data de Reprovação']

In [130]:
# df_final['Data Cadastro Prazo'] = df_final['Data Cadastro Prazo'].dt.strftime("%d/%m/%Y")
# df_final['Data do Prazo'] = df_final['Data do Prazo'].dt.strftime("%d/%m/%Y")
# df_final['Data Inicio Compromisso'] = df_final['Data Inicio Compromisso'].dt.strftime("%d/%m/%Y")
# df_final['Data Conclusão'] = df_final['Data Conclusão'].dt.strftime("%d/%m/%Y")
# df_final['Data Protocolo'] = df_final['Data Protocolo'].dt.strftime("%d/%m/%Y")
# df_final['Data Auditoria Protocolo'] = df_final['Data Auditoria Protocolo'].dt.strftime("%d/%m/%Y")
# df_final['Prazo para Protocolo'] = df_final['Prazo para Protocolo'].dt.strftime("%d/%m/%Y")
# df_final['Data Prazo Automático'] = df_final['Data Prazo Automático'].dt.strftime("%d/%m/%Y")
# df_final['Data Revisão'] = df_final['Data Revisão'].dt.strftime("%d/%m/%Y")
# df_final['Prazo Revisão'] = df_final['Prazo Revisão'].dt.strftime("%d/%m/%Y")
# df_final['Data Cancelamento'] = df_final['Data Cancelamento'].dt.strftime("%d/%m/%Y")
# df_final['Data da contratação'] = df_final['Data da contratação'].dt.strftime("%d/%m/%Y")

In [22]:
df_final.head()

,ID Processo,ID Prazo,Data Cadastro Prazo,Data do Prazo,Hora do Prazo,Tipo de Atividade,Sub Tipo Compromisso,Parte Cliente,Parte Adversa,Número do Processo,...,Tem serviço contratado?,Id serviço,Tipo,Subtipo do serviço,Data da contratação,Hora da contratação,Status do serviço,Status Sugestão alteração/cancelamento,Vinculado ao Recurso?,ID Recurso
0,510525,5328672,2023-02-13,2023-02-15,00:00:00,Prazo,Memoriais,IPIRANGA PRODUTOS DE PETROLEO,ANTONIO MADSON AREIAS DA SILVA,0000581-67.2020.5.08.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
1,843885,5332972,2023-02-13,2023-05-25,11:10:00,Prazo,Contestação - Com Audiência,VIA VAREJO ONLINE,ADELMO AVELINO PEREIRA,0022912-70.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
2,843659,5329501,2023-02-13,2023-03-27,16:00:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ADAILTON SALES DE OLIVEIRA,0022202-50.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
3,843665,5329623,2023-02-13,2023-05-22,08:50:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,GEDEVAL RAMOS DE JESUS FILHO,0021328-65.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
4,843441,5327649,2023-02-13,2023-03-17,10:10:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ELICLEIA BARBOSA DOS SANTOS,0003025-57.2023.8.05.0080,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0


In [30]:
df_final['Data Cadastro Prazo'] = pd.to_datetime(df_final['Data Cadastro Prazo'])
df_final['Data Cadastro Prazo'] = df_final['Data Cadastro Prazo'].dt.strftime('%d/%m/%Y')
df_final['Data Cadastro Prazo'] = pd.to_datetime(['Data Cadastro Prazo'])

C:\Users\victoramarante\AppData\Local\Temp\ipykernel_720\1807432080.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_final['Data Cadastro Prazo'] = pd.to_datetime(df_final['Data Cadastro Prazo'])
C:\Users\victoramarante\AppData\Local\Temp\ipykernel_720\1807432080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Data Cadastro Prazo'] = pd.to_datetime(df_final['Data Cadastro Prazo'])
C:\Users\victoramarante\AppData\Local\Temp\ipykernel_720\1807432080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

ParserError: Unknown string format: Data Cadastro Prazo present at position 0

In [27]:
df_final.head()

,ID Processo,ID Prazo,Data Cadastro Prazo,Data do Prazo,Hora do Prazo,Tipo de Atividade,Sub Tipo Compromisso,Parte Cliente,Parte Adversa,Número do Processo,...,Tem serviço contratado?,Id serviço,Tipo,Subtipo do serviço,Data da contratação,Hora da contratação,Status do serviço,Status Sugestão alteração/cancelamento,Vinculado ao Recurso?,ID Recurso
0,510525,5328672,2023-02-13,2023-02-15,00:00:00,Prazo,Memoriais,IPIRANGA PRODUTOS DE PETROLEO,ANTONIO MADSON AREIAS DA SILVA,0000581-67.2020.5.08.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
1,843885,5332972,2023-02-13,2023-05-25,11:10:00,Prazo,Contestação - Com Audiência,VIA VAREJO ONLINE,ADELMO AVELINO PEREIRA,0022912-70.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
2,843659,5329501,2023-02-13,2023-03-27,16:00:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ADAILTON SALES DE OLIVEIRA,0022202-50.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
3,843665,5329623,2023-02-13,2023-05-22,08:50:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,GEDEVAL RAMOS DE JESUS FILHO,0021328-65.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
4,843441,5327649,2023-02-13,2023-03-17,10:10:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ELICLEIA BARBOSA DOS SANTOS,0003025-57.2023.8.05.0080,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0


In [28]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2097
Data columns (total 59 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   ID Processo                             2091 non-null   int64         
 1   ID Prazo                                2091 non-null   int64         
 2   Data Cadastro Prazo                     2091 non-null   datetime64[ns]
 3   Data do Prazo                           2091 non-null   datetime64[ns]
 4   Hora do Prazo                           2091 non-null   object        
 5   Tipo de Atividade                       2091 non-null   object        
 6   Sub Tipo Compromisso                    2091 non-null   object        
 7   Parte Cliente                           2091 non-null   object        
 8   Parte Adversa                           2091 non-null   object        
 9   Número do Processo                      2091 non-nul

In [21]:
df_final.head()

,ID Processo,ID Prazo,Data Cadastro Prazo,Data do Prazo,Hora do Prazo,Tipo de Atividade,Sub Tipo Compromisso,Parte Cliente,Parte Adversa,Número do Processo,...,Tem serviço contratado?,Id serviço,Tipo,Subtipo do serviço,Data da contratação,Hora da contratação,Status do serviço,Status Sugestão alteração/cancelamento,Vinculado ao Recurso?,ID Recurso
0,510525,5328672,2023-02-13,2023-02-15,00:00:00,Prazo,Memoriais,IPIRANGA PRODUTOS DE PETROLEO,ANTONIO MADSON AREIAS DA SILVA,0000581-67.2020.5.08.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
1,843885,5332972,2023-02-13,2023-05-25,11:10:00,Prazo,Contestação - Com Audiência,VIA VAREJO ONLINE,ADELMO AVELINO PEREIRA,0022912-70.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
2,843659,5329501,2023-02-13,2023-03-27,16:00:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ADAILTON SALES DE OLIVEIRA,0022202-50.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
3,843665,5329623,2023-02-13,2023-05-22,08:50:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,GEDEVAL RAMOS DE JESUS FILHO,0021328-65.2023.8.05.0001,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0
4,843441,5327649,2023-02-13,2023-03-17,10:10:00,Prazo,Contestação - Com Audiência,Via Varejo S/A,ELICLEIA BARBOSA DOS SANTOS,0003025-57.2023.8.05.0080,...,Não,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Nao,0


In [ ]:
# df_final['Data Cadastro Prazo'] = pd.to_datetime(df['Data Cadastro Prazo'], format='%Y-%m-%d', errors='coerce')
# df_final['Data Cadastro Prazo'] = df['Data Cadastro Prazo'].dt.strftime('%d/%m/%Y')

In [ ]:
for i in colunas_datas:
    print(i)

In [ ]:
df_final.to_excel(f'BASE_TRATADA_AGENDADOS_{hoje}.xlsx', index=False, engine='openpyxl')

In [ ]:
tratamento_prazos_diarios('base1302_agendados.xlsx',
    'centro_de_custo.xlsx')